In [23]:
'''
  code by Minho Ryu @bzantium
  
'''
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

# FFNN Parameter
n_hidden = 128
num_classes = 2 # 0 or 1

# sentences
sentences = ["cnn is good for vision","it can classify images well", "object detection techs is matured enough", 
             "deep learning for nlp is growing fast","it can classify words well", "i can build nlp model"]
labels = [1,1,1,0,0,0] # 1 is vision, 0 is nlp.

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

inputs = []
for sen in sentences:
    _input = np.zeros([vocab_size])
    _input[[word_dict[n] for n in sen.split()]] = 1
    inputs.append(_input) # ONE-HOT Bag of Words

outputs = []
for out in labels:
    outputs.append(np.eye(num_classes)[out]) # ONE-HOT : To using Tensor Softmax Loss function
    
class FFNN(object):
    def __init__(self, sess, vocab_size, n_hidden, num_classes):
        self.sess = sess
        self.vocab_size = vocab_size
        self.n_hidden = n_hidden
        self.num_classes = num_classes
        self._build_model()
    
    def _build_model(self):
        self.X = tf.placeholder(tf.float32, [None, self.vocab_size])
        self.Y = tf.placeholder(tf.int32, [None, self.num_classes])
        W_h = tf.Variable(tf.random_uniform([self.vocab_size, self.n_hidden]))
        b_h = tf.Variable(tf.constant(0.1, shape=[self.n_hidden]))
        W_o = tf.Variable(tf.random_uniform([self.n_hidden, self.num_classes]))
        b_o = tf.Variable(tf.constant(0.1, shape=[self.num_classes]))
        
        hidden = tf.nn.xw_plus_b(self.X, W_h, b_h)
        hidden = tf.nn.relu(hidden)
        logits = tf.nn.xw_plus_b(hidden, W_o, b_o)
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(0.001).minimize(self.cost)
        hypothesis = tf.nn.softmax(logits)
        self.predictions = tf.argmax(hypothesis, 1)

        tf.global_variables_initializer().run()
    
    def train(self, inputs, labels):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: inputs, self.Y: labels})
     
    def predict(self, inputs):
        return self.sess.run([self.predictions], feed_dict={self.X: inputs})
        
        
# Training
run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth=True
with tf.Session(config=run_config) as sess:
    model = FFNN(sess, vocab_size, n_hidden, num_classes)
    for epoch in range(5000):
        loss, _ = model.train(inputs, outputs)
        if (epoch + 1)%1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    
    # Test
    test_text = ['vision techs is matured', 'deep learning can build nlp model']
    tests = []
    for text in test_text:
        test = np.zeros([vocab_size])
        test[[word_dict[n] for n in text.split()]] = 1
        tests.append(test)
    result = model.predict(tests)[0]
    
for i, text in enumerate(test_text):
  if result[i] == 0:
      print("\'" + text + "\'", "is about nlp")
  else:
      print("\'"+ text + "\'", "is about vision")

Epoch: 1000 cost = 0.001846
Epoch: 2000 cost = 0.000527
Epoch: 3000 cost = 0.000227
Epoch: 4000 cost = 0.000114
Epoch: 5000 cost = 0.000062
'vision techs is matured' is about vision
'deep learning can build nlp model' is about nlp
